In [1]:
import numpy as np
import pandas as pd

In [5]:
df = pd.read_csv('no_training_all_question_answers.csv')
df

,SEASON,CLUE VALUE,CLUE CATEGORY,QUESTION,OUR MODEL_ANSWER,REAL ANSWER,CORRECT,CONTEXT USED
0,34,200,REJECTED GREETING CARDS,"\""Have fun in"" this country! ""But remember the...",against the advice of the U.S. Department of S...,Afghanistan,False,"Global Relief Foundation . In November 2001, d..."
1,34,400,REJECTED GREETING CARDS,"\""It's OK, applicant! You didn't want to go to...",University of Houston,Stanford,False,Lauro Cruz . Born in Beaumont to Manuel Cruz a...
2,34,600,REJECTED GREETING CARDS,"\""Tummyache? Next time, take a close look at"" ...",NaN,clams,False,No Context
3,34,800,REJECTED GREETING CARDS,"\""Nice surgery! You finally vented"" this soft,...",The spleen,spleen,True,Splenic injury . The spleen is an organ in the...
4,34,1000,REJECTED GREETING CARDS,"\""Sorry you two had"" these 14-letter ""differen...","""irreconcilable differences"",",irreconcilable,True,"Tony Parker . On 17 November 2010, Longoria fi..."
...,...,...,...,...,...,...,...,...
16310,16,100,COUNTRIES BY COLLEGE,"The University of Sunderland, the University o...",University of Northumbria at Newcastle.,England,False,Stephen Newton (artist) . Newton was born in G...
16311,16,200,COUNTRIES BY COLLEGE,"The Emile Cohl School, the University of Toulo...",NaN,France,False,No Context
16312,16,300,COUNTRIES BY COLLEGE,"Chiba University, Waseda University, Fukuoka J...",Tokai University,Japan,False,Tokai University Fukuoka Junior College . Toka...
16313,16,200,HIRSCHFELD,Broadway's Diamond Lil; come up & see her some...,Diamond Lil (play,Mae West,False,Diamond Lil (play) . Diamond Lil is a 1928 pla...


In [25]:
# General stats

# s34 aired September 11, 2017
# s16 aired December 6, 1999
seasons = df['SEASON'].value_counts()
print(seasons)

# total questions answered correctly/incorrectly
df['CORRECT'].value_counts(normalize=True)

34    13362
16     2953
Name: SEASON, dtype: int64


False    0.822924
True     0.177076
Name: CORRECT, dtype: float64

In [16]:
# Investigate answers that were empty strings
nan_rows = df[df['OUR MODEL_ANSWER'].isna()]
nan_rows

percent_nan_rows = df['OUR MODEL_ANSWER'].isna().mean() * 100
print(f'percentage of answers that were empty strings aka Nan: {percent_nan_rows}')

percentage of answers that were empty strings aka Nan 20.091939932577382


In [20]:
# Investigate question length

average_words = df['QUESTION'].str.split().apply(len).mean()
average_words_true = df[df['CORRECT'] == True]['QUESTION'].str.split().apply(len).mean()
average_words_false = df[df['CORRECT'] == False]['QUESTION'].str.split().apply(len).mean()

print(f'The average number of words in a question is: {average_words:.3f}')
print(f"The average number of words in a question for CORRECT=True is: {average_words_true:.3f}")
print(f"The average number of words in a question for CORRECT=False is: {average_words_false:.3f}")

The average number of words in a question is: 14.951
The average number of words in a question for CORRECT=True is: 15.489
The average number of words in a question for CORRECT=False is: 14.835


In [3]:
df = pd.read_csv('jeopardy_clue_dataset-master/seasons/season34.tsv', sep='\t')
df

,round,clue_value,daily_double_value,category,comments,answer,question,air_date,notes
0,1,200,0,REJECTED GREETING CARDS,NaN,"\""Have fun in"" this country! ""But remember the...",Afghanistan,2017-09-11,NaN
1,1,400,0,REJECTED GREETING CARDS,NaN,"\""It's OK, applicant! You didn't want to go to...",Stanford,2017-09-11,NaN
2,1,600,0,REJECTED GREETING CARDS,NaN,"\""Tummyache? Next time, take a close look at"" ...",clams,2017-09-11,NaN
3,1,800,0,REJECTED GREETING CARDS,NaN,"\""Nice surgery! You finally vented"" this soft,...",spleen,2017-09-11,NaN
4,1,1000,0,REJECTED GREETING CARDS,NaN,"\""Sorry you two had"" these 14-letter ""differen...",irreconcilable,2017-09-11,NaN
...,...,...,...,...,...,...,...,...,...
13357,2,800,0,THE U.S. CONGRESS WITH NANCY PELOSI,(Nancy Pelosi: I'm House Democratic Leader Nan...,(Nancy Pelosi reads the clue.) While Congress ...,1812,2018-07-27,NaN
13358,2,1200,0,THE U.S. CONGRESS WITH NANCY PELOSI,(Nancy Pelosi: I'm House Democratic Leader Nan...,(Nancy Pelosi reads the clue.) Nonvoting membe...,Puerto Rico,2018-07-27,NaN
13359,2,1600,0,THE U.S. CONGRESS WITH NANCY PELOSI,(Nancy Pelosi: I'm House Democratic Leader Nan...,(Nancy Pelosi reads the clue.) In February 201...,DACA,2018-07-27,NaN
13360,2,2000,0,THE U.S. CONGRESS WITH NANCY PELOSI,(Nancy Pelosi: I'm House Democratic Leader Nan...,"(Nancy Pelosi reads the clue.) In 2009, I prou...",Shirley Chisholm,2018-07-27,NaN
